A notebook to illustrate/test `kmod.mctest.SC_UME`.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import kmod
import kgof
import kgof.goftest as gof
# submodules
from kmod import data, density, kernel, util
from kmod import mctest as mct
import matplotlib
import matplotlib.pyplot as plt
import autograd.numpy as np
import scipy.stats as stats

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## A simple 1d Gaussian problem

Two models $P = \mathcal{N}(\mu_p, \sigma_p^2)$ and $Q = \mathcal{N}(\mu_q, \sigma^2_q)$. The data generating distribution is $R = \mathcal{N}(0, 1)$.

    H_0: P, Q are equally good
    H_1: Q is better for approximating R

In [ ]:
mp, varp = 1.5, 1
# q cannot be the true model. 
# That violates our assumption and the asymptotic null distribution
# does not hold.
mq, varq = 1.0, 1

# draw some data
n = 600 # sample size
seed = 8
with util.NumpySeedContext(seed=seed):
    X = np.random.randn(n, 1)*varp**0.5 + mp
    Y = np.random.randn(n, 1)*varq**0.5 + mq
    Z = np.random.randn(n, 1)
    
    datap = data.Data(X)
    dataq = data.Data(Y)
    datar = data.Data(Z)

In [ ]:
# plot the data
plt.figure(figsize=(8, 4))
plt.hist(X, color='r', alpha=0.6, normed=True, label='X')
plt.hist(Y, color='b', alpha=0.6, normed=True, label='Y')
plt.hist(Z, color='k', alpha=0.8, normed=True, label='Z')
plt.title('H1: Y is closer to Z')
plt.legend()

### Use random test locations and median heuristic for the Gaussian widths

In [ ]:
# hyperparameters of the test
medxz = util.meddistance(np.vstack((X, Z)), subsample=1000)
medyz = util.meddistance(np.vstack((Y, Z)), subsample=1000)
k = kernel.KGauss(sigma2=medxz**2)
l = kernel.KGauss(sigma2=medyz**2)
# 2 sets of test locations
J = 2
# Jp = J
# Jq = J
V = util.fit_gaussian_draw(X, J, seed=seed+2)
# W = util.fit_gaussian_draw(Y, Jq, seed=seed+3)
W = V

In [ ]:
# construct a UME test
alpha = 0.01 # significance level 
scume = mct.SC_UME(datap, dataq, k, l, V, W, alpha=alpha)
scume.perform_test(datar)

### Optimize the test locations and Gaussian widths

Optimize two sets V, W of test locations and two Gaussian widths. Each set is optimized separately by maximizing the power criterion of its corresponding two-sample test problem. Specifically, V is optimized on the power criterion of UME(P, R), and W is optimized on the power criterion of UME(Q,R).

Optimizing the two sets in this way does not necessarily give parameters which maximize the test power of the three-sample test that we consider. 

In [ ]:
# split the data into training/test sets
[(datptr, datpte), (datqtr, datqte), (datrtr, datrte)] = \
    [D.split_tr_te(tr_proportion=0.3, seed=85) for D in [datap, dataq, datar]]
Xtr, Ytr, Ztr = [D.data() for D in [datptr, datqtr, datrtr]]

In [ ]:
# initialize optimization parameters.
# Initialize the Gaussian widths with the median heuristic
medxz = util.meddistance(np.vstack((Xtr, Ztr)), subsample=1000)
medyz = util.meddistance(np.vstack((Ytr, Ztr)), subsample=1000)
gwidth0p = medxz**2
gwidth0q = medyz**2

# numbers of test locations in V, W
J = 2
Jp = J
Jq = J

# pick a subset of points in the training set for V, W
Xyztr = np.vstack((Xtr, Ytr, Ztr))
VW = util.subsample_rows(Xyztr, Jp+Jq, seed=73)
V0 = VW[:Jp, :]
W0 = VW[Jp:, :]

# optimization options
opt_options = {
    'max_iter': 100,
    'reg': 1e-4,
    'tol_fun': 1e-6,
    'locs_bounds_frac': 100,
    'gwidth_lb': None,
    'gwidth_ub': None,
}

umep_params, umeq_params = mct.SC_GaussUME.optimize_2sets_locs_widths(
    datptr, datqtr, datrtr, V0, W0, gwidth0p, gwidth0q, 
    **opt_options)
(V_opt, gw2p_opt, opt_infop) = umep_params
(W_opt, gw2q_opt, opt_infoq) = umeq_params
k_opt = kernel.KGauss(gw2p_opt)
l_opt = kernel.KGauss(gw2q_opt)

In [ ]:
opt_infoq

In [ ]:
# plot the data and the learned locations
plt.figure(figsize=(8, 4))
plt.hist(Xtr, color='r', alpha=0.6, normed=True, label='X')
plt.hist(Ytr, color='b', alpha=0.6, normed=True, label='Y')
plt.hist(Ztr, color='k', alpha=0.8, normed=True, label='Z')
for v in V_opt:
    plt.plot(v[0], 0, '^', color='magenta', markersize=40)
for w in W_opt:
    plt.plot(w[0], 0, '*', color='green', markersize=40)
    
plt.title('H1: Y is closer to Z')
plt.legend()

In [ ]:
# construct a UME test
alpha = 0.01 # significance level 
scume_opt2 = mct.SC_UME(datpte, datqte, k_opt, l_opt, V_opt, W_opt, alpha=alpha)
scume_opt2.perform_test(datrte)

In [ ]:
gw2p_opt, gw2q_opt

### Optimize the power criterion of the three-sample test

Assume that UME(P, R) and UME(Q, R) share the same set V of J test locations, and the same Gaussian kernel. Optimize V and the Gaussian bandwidth by maximizing the test power criterion of the three-sample test.

In [ ]:
# split the data into training/test sets
[(datptr, datpte), (datqtr, datqte), (datrtr, datrte)] = \
    [D.split_tr_te(tr_proportion=0.4, seed=85) for D in [datap, dataq, datar]]
Xtr, Ytr, Ztr = [D.data() for D in [datptr, datqtr, datrtr]]
Xyztr = np.vstack((Xtr, Ytr, Ztr))

In [ ]:
# initialize optimization parameters.
# Initialize the Gaussian widths with the median heuristic
medxyz = util.meddistance(Xyztr, subsample=1000)
gwidth0 = medxyz**2

# numbers of test locations in V = W
J = 2

# pick a subset of points in the training set for V, W
Xyztr = np.vstack((Xtr, Ytr, Ztr))
V0 = util.subsample_rows(Xyztr, J, seed=75)

# optimization options
opt_options = {
    'max_iter': 100,
    'reg': 1e-4,
    'tol_fun': 1e-6,
    'locs_bounds_frac': 100,
    'gwidth_lb': None,
    'gwidth_ub': None,
}
V_opt, gw2_opt, opt_result = mct.SC_GaussUME.optimize_3sample_criterion(
    datptr, datqtr, datrtr, V0, gwidth0, **opt_options)    
k_opt = kernel.KGauss(gw2_opt)

display(opt_result)

In [ ]:
# construct a UME test
alpha = 0.01 # significance level 
scume_opt3 = mct.SC_UME(datpte, datqte, k_opt, k_opt, V_opt, V_opt, alpha=alpha)
scume_opt3.perform_test(datrte)

In [ ]:
# plot the data and the learned locations
plt.figure(figsize=(8, 4))
plt.hist(Xtr, color='r', alpha=0.6, normed=True, label='X')
plt.hist(Ytr, color='b', alpha=0.6, normed=True, label='Y')
plt.hist(Ztr, color='k', alpha=0.8, normed=True, label='Z')
for v in V_opt:
    plt.plot(v[0], 0, '^', color='magenta', markersize=40)
    
plt.title('H1: Y is closer to Z')
plt.legend()

The learned locations are supposed to show where Q fits (to Z) better than P does.

-----------

# Debugging

Normal usage will not need the following code. The following code is here for checking the implementation during the development.

Check the asymptotic distribution of the SC_UME statistic.

In [ ]:
def gen_test_samples(n, seed):
    """
    Return datap, dataq, datar
    """
    mp, varp = 0.5, 1
    mq, varq = 1, 1

    # draw some data
    
    with util.NumpySeedContext(seed=seed):
        X = np.random.randn(n, 1)*varp**0.5 + mp
        Y = np.random.randn(n, 1)*varq**0.5 + mq
        Z = np.random.randn(n, 1)

        datap = data.Data(X)
        dataq = data.Data(Y)
        datar = data.Data(Z)
    return datap, dataq, datar

In [ ]:
seed = 1003
n = 300 # sample size
datap, dataq, datar = gen_test_samples(n, seed)
X, Y, Z = [a.data() for a in [datap, dataq, datar]]

In [ ]:
# plot the data
plt.figure(figsize=(8, 4))
plt.hist(X, color='r', alpha=0.6, normed=True, label='X')
plt.hist(Y, color='b', alpha=0.6, normed=True, label='Y')
plt.hist(Z, color='k', alpha=0.8, normed=True, label='Z')
plt.title('H1: Y is closer to Z')
plt.legend()

In [ ]:
# hyperparameters of the test
medxz = util.meddistance(np.vstack((X, Z)), subsample=1000)
medyz = util.meddistance(np.vstack((Z, Y)), subsample=1000)
k = kernel.KGauss(sigma2=medxz**2)
l = kernel.KGauss(sigma2=medyz**2)

# 2 sets of test locations
J = 1
Jp = J
Jq = J


In [ ]:
# pick a subset of points from the data for the two sets of locations
pool3J = np.vstack((X[:J, :], Y[:J, :], Z[:J, :]))
X, Y, Z = [np.delete(a, range(J), 0) for a in [X, Y, Z]]
datp, datq, datr = [data.Data(a) for a in [X, Y, Z]]
assert X.shape[0] == Y.shape[0]
assert Y.shape[0] == Z.shape[0]
assert Z.shape[0] == n-J
assert datp.sample_size() == n-J
assert datq.sample_size() == n-J
assert datr.sample_size() == n-J

# use two sets of locations: V and W
VW = util.subsample_rows(pool3J, 2*J, seed+1)
# add a little noise to the locations. 
# Remember the locations have to be drawn from a distribution with a density
XYZ = np.vstack((X, Y, Z))
# VW = VW + np.random.randn(2*J, X.shape[1])*np.mean(np.std(XYZ, axis=0))
with util.NumpySeedContext(seed=seed+8):
    VW = VW + np.random.randn(2*J, X.shape[1])*np.mean(np.std(XYZ, axis=0))

V = VW[:J, :]
W = VW[J:, :]


In [ ]:
# V = util.fit_gaussian_draw(X, Jp, seed=seed+2)
# W = util.fit_gaussian_draw(Y, Jq, seed=seed+3)

In [ ]:
# number of times to create a new problem (draw new samples)
trials = 300
null_stats = np.zeros(trials)
alpha = 0.05 # significance level 

for t in range(trials):
    datap, dataq, datar = gen_test_samples(n, seed=(t*17)%(2**31))
    # create a UME test
    
    scume = mct.SC_UME(datap, dataq, k, l, V, W, alpha=alpha)
    null_stats[t] = scume.compute_stat(datar)

# use the data in the last trial to perform test
results = scume.perform_test(datar)

display(results)

In [ ]:
# get the parameters of the asymptotic null distribution
_, var_h0 = scume.get_H1_mean_variance(datar, return_variance=True)
dom =  np.linspace(np.min(null_stats)-1, np.max(null_stats)+2, 300)
ph0_values = stats.norm.pdf(dom, loc=0, scale=var_h0**0.5)

In [ ]:
# histogram of the null stats
plt.figure(figsize=(8, 5))

plt.plot(dom, ph0_values, 'r-', label='Asymp. null dist.')
plt.hist(null_stats, label='Empirical ground truth', alpha=0.7, bins=15, normed=True);
# plt.hist(sim_stats, label='Asymptotic', alpha=0.7, bins=15, normed=True)
plt.legend()

When $H_0$ is true, the asymptotic null distribution is expected to be to the right of the empirically obtained statistics. This means that we will have type-I error which is lower than $\alpha$, but lose a bit of test power.